# <center>Luxury apartment </center>

Ho riscontrato vari problemi con la webapp per la strasformazione da SVG a Lines.
Penso che il problema nasca da Illustrator ma non riesco a trovare il motivo scatenante (il programma suggerito Inkscape mi forniva un risultato estreamemente scadente).
In particolar modo il file delle porte non viene visualizzato mentre le finestre  causano un crash del programma.
Il file dei muri esterni risulta un arrey decisamente troppo lungo e i muri vengono in più punti "tondi" mentre i muri interni risultano tutti disallineati.

!["Planimetria"](img/luxuryApartments.jpg)
!["vista_area"](img/vistaAerea.png)
!["vista_laterale.png"](img/vistaLaterale.png)


In [1]:
from pyplasm import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.00460127321299 seconds


In [2]:
""" intRGBColor Web copied function to create colors from RGB
@param values: array of RGB values
@return: the color
"""
def intRGBColor(values):
        return Color4f([ values[0]/255.0,
                         values[1]/255.0,
                         values[2]/255.0,
                         1.0])
    
SILVER = intRGBColor([192,192,192])
WOOD = intRGBColor([130, 82, 1])

In [3]:
"""Function to read external file"""
def readFile(file_name):
    points = []
    i = 0
    indexes = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            points.append([float(row[0]), float(row[1])])
            points.append([float(row[2]), float(row[3])])
            i += 2
            indexes.append([i-1, i])
    return S([1])([-1])(MKPOL([points, indexes, None]))

In [4]:
"""constant variable"""

#Path
pathExtWalls = ".\lines\esterno.lines"
pathIntWalls = ".\lines\interno.lines"
pathDoors = ".\lines\porte.lines"
pathWindows = ".\lines\esterno1.lines"

#var
extWallsThickness = 5
intWallsThickness = 2
houseHeight = 40
windowsHeight = 10
windowsHeightFromFloor = 15
doorsHeight = 25

In [5]:
"""create the final structure"""

def LuxuryApartment():
    # External Walls
    extWallsHPC = readFile(pathExtWalls)
    extWallsThick = OFFSET([extWallsThickness,extWallsThickness])(extWallsHPC)
    extWallsTall = PROD([extWallsThick, QUOTE([houseHeight])])
    
    # Internal Walls
    internalWallsHPC = readFile(pathIntWalls)
    internalWallsThick = OFFSET([intWallsThickness,intWallsThickness])(internalWallsHPC)
    internalWallDiffered = DIFF([internalWallsThick, extWallsThick])
    internalWallsTall = PROD([internalWallDiffered, QUOTE([houseHeight])])
    
    # Windows
    windowsHPC = readFile(pathWindows)
    windowsThick = OFFSET([extWallsThickness, extWallsThickness])(windowsHPC)
    windowsTall = PROD([windowsThick, QUOTE([windowsHeight])])
    windowsFromFloor = STRUCT([T(3)(windowsHeightFromFloor), windowsTall])
    
    extWallsWindows = DIFF([extWallsTall, windowsFromFloor])
    
    #Doors
    doorsHPC = readFile(pathDoors)
    
    #Doors External
    doorsExtThick = OFFSET([extWallsThickness+1, extWallsThickness+1])(doorsHPC)
    doorsExternalTall = PROD([doorsExtThick, QUOTE([doorsHeight])])
    
    extWallsWindowsDoors = DIFF([extWallsWindows, doorsExternalTall])
    extWallsWindowsDoorsColored = COLOR(RED)(extWallsWindowsDoors)
    
    #Doors Internal
    doorsIntThick = OFFSET([intWallsThickness+1, intWallsThickness+1])(doorsHPC)
    doorsIntTall = PROD([doorsIntThick, QUOTE([doorsHeight])])
    
    intWallsDoors = DIFF([internalWallsTall, doorsIntTall])
    intWallsDoorsColored = COLOR(BLUE)(intWallsDoors)
    
    floor = COLOR(BROWN)(SOLIDIFY(extWallsHPC))
    
    walls = STRUCT([floor, extWallsWindowsDoorsColored, intWallsDoorsColored])
        
    return walls

In [6]:
VIEW(LuxuryApartment())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000AC957E0> >